Criação de um modelo supervisionado de classificação para prever se um pedido será entregue no prazo
para isso, precisa ser feito a classificação de pedidos entregues no prazo vs atrasados

## Importações

In [1]:
# Importar bibliotecas principais para análise e modelagem
import pandas as pd
import numpy as np

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


## Carregando o dataset

In [2]:
# Carregar a tabela consolidada criada no ETL
df = pd.read_parquet("/home/felipe/Desktop/Projetos Free ml/Brazilian_E-Commerce/data/processed/orders_table.parquet")

# Visualizar primeiras linhas
df.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,product_category_name_english,delivery_time_days,estimated_time_days,delay,total_price
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,8.0,13.0,9350.0,maua,SP,housewares,8.0,15,0,38.71
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,8.0,13.0,9350.0,maua,SP,housewares,8.0,15,0,38.71
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,8.0,13.0,9350.0,maua,SP,housewares,8.0,15,0,38.71
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,47813,...,13.0,19.0,31570.0,belo horizonte,SP,perfumery,13.0,19,0,141.46
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,19.0,21.0,14840.0,guariba,SP,auto,9.0,26,0,179.12


## Seleção de features e target

Não selecionamos todas as features por enquanto, mas podemos ir adicionando mais
essas são as mais relevantes até então

In [3]:
def create_features_and_target(df):
    """
    Cria a variável alvo e seleciona features mais relevantes.
    """
    # Converter as colunas de data para o tipo datetime
    df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
    df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])
    df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])

    # Calcular a diferença de dias entre a entrega real e a estimada (nosso target)
    df['delivery_diff'] = (df['order_delivered_customer_date'] - df['order_estimated_delivery_date']).dt.days
    df['is_late'] = (df['delivery_diff'] > 0).astype(int)

    # Calcular o tempo de aprovação do pagamento em dias
    df['payment_approval_time_days'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / (60*60*24)
    # Calcular o tempo de entrega para a transportadora em dias
    df['carrier_delivery_time_days'] = (df['order_delivered_carrier_date'] - df['order_approved_at']).dt.total_seconds() / (60*60*24)

    # Features que fazem mais sentido para a previsão
    selected_features = [
        'customer_state',               # Estado do cliente pode influenciar o tempo de entrega
        'product_category_name_english',  # Categoria do produto pode ter diferentes logísticas
        'price',                        # Preço pode estar relacionado à urgência ou tipo de produto
        'freight_value',                # O valor do frete está diretamente relacionado ao serviço de entrega
        'payment_installments',         # Número de parcelas pode ser um proxy para o valor do produto
        'review_score',                 # Score de avaliação do cliente pode ter relação com o atraso
        'seller_state',                 # A localização do vendedor afeta a distância do frete
        'product_weight_g',             # O peso do produto afeta o frete
        'payment_approval_time_days',   # Tempo de aprovação afeta o início da entrega
        'carrier_delivery_time_days'    # Tempo de envio para a transportadora
    ]

    target = 'is_late'
    
    # Remover linhas com valores nulos nas features selecionadas
    df_cleaned = df.dropna(subset=selected_features + [target])

    return df_cleaned[selected_features + [target]]

# Executar a função e separar os dados
df_processed = create_features_and_target(df)
X = df_processed.drop('is_late', axis=1)
y = df_processed['is_late']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape dos dados de treino:", X_train.shape)
print("Shape dos dados de teste:", X_test.shape)

Shape dos dados de treino: (91519, 10)
Shape dos dados de teste: (22880, 10)


## Divisão em treino e teste

In [4]:
# Dividir os dados em treino e teste (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Tamanho treino:", X_train.shape)
print("Tamanho teste:", X_test.shape)


Tamanho treino: (91519, 10)
Tamanho teste: (22880, 10)


## Variaveis categóricas e numéricas

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114399 entries, 0 to 119142
Data columns (total 10 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   customer_state                 114399 non-null  object 
 1   product_category_name_english  114399 non-null  object 
 2   price                          114399 non-null  float64
 3   freight_value                  114399 non-null  float64
 4   payment_installments           114399 non-null  float64
 5   review_score                   114399 non-null  float64
 6   seller_state                   114399 non-null  object 
 7   product_weight_g               114399 non-null  float64
 8   payment_approval_time_days     114399 non-null  float64
 9   carrier_delivery_time_days     114399 non-null  float64
dtypes: float64(7), object(3)
memory usage: 9.6+ MB


verificando se há valores faltantes para não atraplhar a normalização e treinamento

In [6]:
X.isnull().sum().sort_values(ascending=False)


customer_state                   0
product_category_name_english    0
price                            0
freight_value                    0
payment_installments             0
review_score                     0
seller_state                     0
product_weight_g                 0
payment_approval_time_days       0
carrier_delivery_time_days       0
dtype: int64

Pré processamento com pipeline e normalização 

In [8]:
# Definir as features numéricas e categóricas
numeric_features = ['price', 'freight_value', 'payment_installments', 'review_score', 'product_weight_g', 'payment_approval_time_days', 'carrier_delivery_time_days']
categorical_features = ['customer_state', 'product_category_name_english', 'seller_state']

# Criar o pré-processador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

## Modelo baseline (Regressão Logística)

In [9]:
# Treinar o modelo de Regressão Logística
from sklearn.linear_model import LogisticRegression

log_reg_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

log_reg_pipeline.fit(X_train, y_train)
y_pred_log_reg = log_reg_pipeline.predict(X_test)

# Avaliar o modelo
print("== Regressão Logística ==")
print(classification_report(y_test, y_pred_log_reg))

== Regressão Logística ==
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     21431
           1       0.67      0.12      0.20      1449

    accuracy                           0.94     22880
   macro avg       0.81      0.56      0.58     22880
weighted avg       0.93      0.94      0.92     22880



## Árvore de decisão

In [10]:
# Treinar o modelo de Árvore de Decisão
from sklearn.tree import DecisionTreeClassifier

dt_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

dt_pipeline.fit(X_train, y_train)
y_pred_dt = dt_pipeline.predict(X_test)

# Avaliar o modelo
print("== Árvore de Decisão ==")
print(classification_report(y_test, y_pred_dt))

== Árvore de Decisão ==
              precision    recall  f1-score   support

           0       0.96      0.96      0.96     21431
           1       0.41      0.41      0.41      1449

    accuracy                           0.92     22880
   macro avg       0.68      0.68      0.68     22880
weighted avg       0.93      0.92      0.92     22880



## Random Forest

In [11]:
# Treinar o modelo de Random Forest
from sklearn.ensemble import RandomForestClassifier

rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_test)

# Avaliar o modelo
print("== Random Forest ==")
print(classification_report(y_test, y_pred_rf))

== Random Forest ==
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     21431
           1       0.86      0.28      0.42      1449

    accuracy                           0.95     22880
   macro avg       0.91      0.64      0.70     22880
weighted avg       0.95      0.95      0.94     22880



## Comparação dos modelos

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calcular as métricas para cada modelo
log_reg_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_log_reg),
    'Precision': precision_score(y_test, y_pred_log_reg, zero_division=0),
    'Recall': recall_score(y_test, y_pred_log_reg, zero_division=0),
    'F1-Score': f1_score(y_test, y_pred_log_reg, zero_division=0)
}

dt_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_dt),
    'Precision': precision_score(y_test, y_pred_dt, zero_division=0),
    'Recall': recall_score(y_test, y_pred_dt, zero_division=0),
    'F1-Score': f1_score(y_test, y_pred_dt, zero_division=0)
}

rf_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_rf),
    'Precision': precision_score(y_test, y_pred_rf, zero_division=0),
    'Recall': recall_score(y_test, y_pred_rf, zero_division=0),
    'F1-Score': f1_score(y_test, y_pred_rf, zero_division=0)
}

# Criar um DataFrame para a comparação
comparison_df = pd.DataFrame({
    'Logistic Regression': log_reg_metrics,
    'Decision Tree': dt_metrics,
    'Random Forest': rf_metrics
})

print("Tabela de Comparação de Modelos:")
print(comparison_df.T)

Tabela de Comparação de Modelos:
                     Accuracy  Precision    Recall  F1-Score
Logistic Regression  0.940472   0.673307  0.116632  0.198824
Decision Tree        0.924956   0.407713  0.408558  0.408135
Random Forest        0.951573   0.857442  0.282264  0.424714


## interpretação

- Primeiramente, esse dataset é desbalanceado, pois temos mais valores de "chegou no prazo" ou '0' do que "atrasado" ou '1'.

- Então fica facil para o modelo prever as entregas que chegarão no prazo haja visto que isso é maioria

- portanto, para cada modelo, valores acuracia se mostram bem excelentes (0.92 -> 0.95), mas isso não quer dizer muita coisa pois existe esse desbalanceamento.

- A precisão por outro lado, se mostrou melhor no random forest, depois regressao logista e por fim arvore de decisão (pessima)

- Já em relação ao recall (das realmente atrasadas quantas o modelo encontrou) não tivemos resultados tão expresivos para cada modelo sendo a regressao logistica a pior (12%), depois random forest e a melhor sendo a arvore de decisão

- O f1-score que seria a relação entre a precisao (quantas o modelo disse que estão atrasadas realmente estao) e o recall (das que realmente sao atrasadas quantas o modelo acertou) se mostrou quase igual entre a arvore de decisao e o random forest sendo o ultimo melhor (0.4081 < 0.4247)

- Ao todo os modelos avaliaram 22.880 casos como indicado pelo support. 21.431 sem atraso vs 1449 com atraso confirmando aquilo que foi falado antes sobre o desbalanceamento do dataset

Aplicando o Synthetic Minority Over-sampling Technique ou SMOTE para corrigir esse desbalanceamento

O SMOTE não apaga os dados da classe majoritária, mas sim cria dados sintéticos da classe minoritária (os pedidos atrasados) para equilibrar o conjunto de dados. Isso força o modelo a aprender o "atraso" em vez de simplesmente ignorá-lo.

## Balanceamento dos dados com SMOTE

In [13]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Separar as features numéricas e categóricas
X_num = X_train[numeric_features]
X_cat = X_train[categorical_features]

# Aplicar o OneHotEncoder para as features categóricas para o SMOTE
preprocessor_for_smote = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)
X_cat_encoded = preprocessor_for_smote.fit_transform(X_train)

# Combinar dados codificados e numéricos
X_train_processed = np.hstack([X_cat_encoded.toarray(), X_num])

# Contar o número de classes antes do SMOTE
print("Contagem de classes antes do SMOTE:", Counter(y_train))

# Aplicar o SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train)

print("Contagem de classes após o SMOTE:", Counter(y_train_resampled))

Contagem de classes antes do SMOTE: Counter({0: 85722, 1: 5797})
Contagem de classes após o SMOTE: Counter({0: 85722, 1: 85722})


## Treinando os modelos novamente com os dados balanceados

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# O pipeline agora não precisa do preprocessor, pois os dados já foram transformados para o SMOTE
# No entanto, a padronização das features numéricas ainda é necessária. 
# Reajustando a lógica de treino para o notebook.

# Treinar Regressão Logística
# É necessário aplicar o StandardScaler nos dados numéricos novamente, pois o SMOTE não o faz.
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_resampled[:, -len(numeric_features):]) # Padroniza os dados numéricos após o SMOTE
X_train_final = np.hstack([X_train_resampled[:, :-len(numeric_features)], X_train_scaled])

X_test_scaled = scaler.transform(X_test[numeric_features])
X_test_final = np.hstack([preprocessor_for_smote.transform(X_test).toarray(), X_test_scaled])

log_reg_model = LogisticRegression(random_state=42, max_iter=1000)
log_reg_model.fit(X_train_final, y_train_resampled)
y_pred_log_reg_smote = log_reg_model.predict(X_test_final)

# Treinar Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_final, y_train_resampled)
y_pred_rf_smote = rf_model.predict(X_test_final)

# O `ColumnTransformer` é melhor usado dentro do `Pipeline`, então vamos seguir a melhor prática para o próximo passo.
# Por enquanto, esta forma manual demonstra o impacto do SMOTE.

/home/felipe/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/felipe/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Comparando os resultados

In [15]:
from sklearn.metrics import classification_report

print("== Regressão Logística (com SMOTE) ==")
print(classification_report(y_test, y_pred_log_reg_smote))

print("\n== Random Forest (com SMOTE) ==")
print(classification_report(y_test, y_pred_rf_smote))

== Regressão Logística (com SMOTE) ==
              precision    recall  f1-score   support

           0       0.98      0.81      0.89     21431
           1       0.21      0.74      0.33      1449

    accuracy                           0.81     22880
   macro avg       0.60      0.78      0.61     22880
weighted avg       0.93      0.81      0.85     22880


== Random Forest (com SMOTE) ==
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     21431
           1       0.69      0.39      0.50      1449

    accuracy                           0.95     22880
   macro avg       0.83      0.69      0.74     22880
weighted avg       0.94      0.95      0.94     22880

